In [ ]:
import cv2
import numpy as np
import ctypes
from ctypes import *
from scipy import misc
from time import sleep
import csv
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from instrumental.drivers.cameras import uc480


In [ ]:
## Step 2: Trail run to troubleshoot camera linking, finding zeroth order point location, and making sure stuff runs. 
# The biggest challenge here is probably finding the 0th order and making sure the location is stable.
# If we are working in an unstable environment, might be worth using cv2 to find the location of 0th order spot.
# However, there are print lines scattered throughout the code to help pinpoint where things go FUBAR.


awareness = ctypes.c_int()
errorCode = ctypes.windll.shcore.GetProcessDpiAwareness(0, ctypes.byref(awareness))
print(awareness.value)
errorCode = ctypes.windll.shcore.SetProcessDpiAwareness(2)
success = ctypes.windll.user32.SetProcessDPIAware()
cdll.LoadLibrary("C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\SDK\\Blink_C_wrapper")
slm_lib = CDLL("Blink_C_wrapper")
cdll.LoadLibrary("C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\SDK\\ImageGen")
image_lib = CDLL("ImageGen")
instruments = uc480.list_instruments()
cam = uc480.UC480_Camera(instruments[0])
cam.start_live_video(framerate = "10Hz")
print("Camera Loaded")
RGB = c_uint(1)
is_eight_bit_image = c_uint(0)
slm_lib.Create_SDK(c_uint(1)) # Initialize SDK, c_unit(1) means true
print("Created SDK")

In [ ]:
## Settings
NumDataPoints = 256
NumRegions = 64 ## If we need regional calibration, set this to 64
### numFrames determines how many times we will take a picture of a set diffraction grating. I recommend 3~5 to get a good average. 
numFrames = 1
PixelValueOne = c_uint(255)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!111
## change this to adjust the frequency of the calibration tweezers 
PixelsPerStripe = c_uint(4)

AI_Intensities = np.zeros([NumRegions,NumDataPoints], np.float32, 'C')
#####  

height = c_uint(slm_lib.Get_Height())
width = c_uint(slm_lib.Get_Width())
depth = c_uint(slm_lib.Get_Depth())
center_x = c_uint(width.value//2)
center_y = c_uint(height.value//2)

success = 0
success = slm_lib.Load_lut("C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\LUT Files\\19x12_8bit_linearVoltage.lut")
print("Load LUT success")
#these arrays hold our data points. 
Image = np.empty([width.value*height.value*3], np.uint8, 'C')
WFC = np.zeros([width.value*height.value*3], np.uint8, 'C')
image_lib.Generate_Solid(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, c_uint(0), RGB)
print("Generate Image success")
slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), c_uint(1))
print("Write Image success")


In [ ]:
## Determine camera settings

REMEMBER TO ADD IN THE WAVEFRONT CORRECTION


millisec = 3
exposure=f'{millisec}ms'

testpixelval = c_uint(255 - 60) # testmax
image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,30,c_uint(NumRegions),RGB)
slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
sleep(0.03)
frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
plt.imshow(frame)



In [ ]:
plt.imshow(frame[100:200,100:200])

In [ ]:
## NOTE TO SELF: WRITE SOME CODE HERE THAT WILL CORRECT FOR TOO LOW/HIGH EXPOSURE
millisec = 3
exposure=f'{millisec}ms'
ymin = 
ymax = 
xmin = 
xmax = 


for region in range(0,NumRegions,1):
    # region = 3
    validregion = True
    print("Region: %d\n", region)
    
    millisec = 3
    exposure=f'{millisec}ms'
    # initial calibration
    ## Test max = 60, Test min = 110
    testpixelval = c_uint(255 - 60) # testmax
    image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
    image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
    slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
    sleep(0.03)
    frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
    testmaxval = np.max(frame[ymin:ymax, xmin:xmax])
    
    
    testpixelval = c_uint(255 - 110) # testmin
    image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
    image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
    slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
    sleep(0.03)
    frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
    testminval = np.max(frame[ymin:ymax, xmin:xmax])
    
    if (10 > testminval or testmaxval < 230):
        millisec = 0.5
    while 10 > testminval or testmaxval < 230:
        if millisec > 10:
            print("error")
            AI_Intensities[region, :] = 0
            validregion = False
        millisec += 0.1
        exposure=f'{millisec}ms'

        testpixelval = c_uint(255 - 60) # testmax
        image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
        image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
        slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
        sleep(0.03)
        frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
        testmaxval = np.max(frame[ymin:ymax, xmin:xmax])
        
        
        testpixelval = c_uint(255 - 110) # testmin
        image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
        image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
        slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
        sleep(0.03)
        frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
        testminval = np.max(frame[ymin:ymax, xmin:xmax])
        
        
    
    if validregion:
        for gray in range(0,NumDataPoints,1):
            print("Gray: %d\r", gray)
            PixelValueTwo = c_uint(255-gray)
            image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
            image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
            slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
            sleep(0.03)
            net = 0
            for frames in range(0, numFrames, 1):
                #Important: exposure_time can be set to your liking, choose one that's best for saturation
                frame = cam.grab_image(timeout='100s', copy=True, exposure_time=exposure)
                numpix = 0 
                #this region has to be handled by you!!! Find the region where the 0th order / 1st order spot is
                #You can average over nearby pixels, take the max value, or something else. 
                #You can find where the desired spot is by passing a diffraction grating to the SLM manually and opening the Thorlabs camera in Python. Then translate camera to an array, find points on array, and put in coords. Code for this should be available above.
                net += np.max(frame[ymin:ymax, xmin:xmax])
            AI_Intensities[region, gray] = net / numFrames
            # this will give you an indicator of your pixel values. make sure there's no over/under saturation!



In [ ]:
plt.imshow(AI_Intensities)

In [ ]:
np.savez_compressed("regionalcalibration.npz", array1=AI_Intensities)

In [ ]:
for regions in range(0,64,1):
    filename = f"Raw{regions}.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for datapoints in range(NumDataPoints):
            writer.writerow([datapoints, AI_Intensities[regions, datapoints]])


In [ ]:
### Step 3: Get your intensity values!
# once you find the location, let's run that loop once more. Note that the camera should still be open, the SLM
# should still be initialized, and we are good to go. IF stuff doesn't run, just try restarting the notebook and running cells from
# top to bottom. For some reason the SLM's shutdown function doesn't work(?) so I've just been restarting the notebook to
# close the SLM. 
NumDataPoints = 256
PixelValueOne = c_uint(255)
PixelsPerStripe = c_uint(4)
NumRegions = 64
AI_Intensities = np.zeros([NumRegions,NumDataPoints], np.float32, 'C')

Regions_Of_Interest = [29,34,42,49,54,60]
for region in Regions_Of_Interest:
    print("Region: %d\n", region)
    for gray in range(0,NumDataPoints,1):
        PixelValueTwo = c_uint(255-gray)
        image_lib.Generate_Stripe(Image.ctypes.data_as(POINTER(c_ubyte)), WFC.ctypes.data_as(POINTER(c_ubyte)), width, height, depth, PixelValueOne, PixelValueTwo, PixelsPerStripe, RGB)
        image_lib.Mask_Image(Image.ctypes.data_as(POINTER(c_ubyte)), width, height,depth,region,c_uint(NumRegions),RGB)
        slm_lib.Write_image(Image.ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)
        # set a 30ms delay to let SLM settle.
        sleep(0.03)
        net = 0
        ##net = np.sum(map[509:512,644:647])
        time = '2.7ms'
        map = cam.grab_image(timeout='100s', copy=True, exposure_time = time)
        #change map[] to desired coordinates of spot. 
        net = np.max(map[200:600, 600:1000]) +net
        
        map = cam.grab_image(timeout='100s', copy=True, exposure_time = time)
        #change map[] to desired coordinates of spot. 
        net = np.max(map[200:600, 600:1000]) +net
        
        net = net / 2
        AI_Intensities[region, gray] = net
        print(net)
## Again, make sure there's no over/undersaturation. If there is, it's time to adjust exposure time or dark lens.
print(AI_Intensities)